In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine


In [2]:
suicide_file = "Resources/master.csv"
happiness_2015 = "Resources/2015.csv"
happiness_2016 = "Resources/2016.csv"
happiness_2017 = "Resources/2017.csv"

In [15]:
suicide_df = pd.read_csv(suicide_file)

hap_2015 = pd.read_csv(happiness_2015)
hap_2016 = pd.read_csv(happiness_2016)
hap_2017 = pd.read_csv(happiness_2017)

hap_2015["year"] = 2015
hap_2016["year"] = 2016
hap_2017["year"] = 2017

hap_2015 = hap_2015[["Country", "Happiness Rank", "Happiness Score", "Economy (GDP per Capita)", \
                    "Family", 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',\
                     'Generosity', 'Dystopia Residual', 'year']]

hap_2016 = hap_2016[["Country", "Happiness Rank", "Happiness Score", "Economy (GDP per Capita)", \
                    "Family", 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual', \
                    'year']]

hap_2017 = hap_2017[["Country", "Happiness.Rank", "Happiness.Score", "Economy..GDP.per.Capita.", \
                    "Family", 'Trust..Government.Corruption.', 'Generosity', 'Dystopia.Residual', \
                    'year']]

hap_2015 = hap_2015.rename(columns={'Country':'country',"Happiness Rank":"happiness_rank", \
                                    "Happiness Score":"happiness", \
                                    "Economy (GDP per Capita)":"economy", "Family": "family", \
                                    'Health (Life Expectancy)':'health', \
                                    'Freedom': 'freedom', 'Trust (Government Corruption)':'trust' ,\
                                    'Generosity':'generosity', 'Dystopia Residual':'dystopia'})

hap_2016 = hap_2016.rename(columns={'Country':'country',"Happiness Rank":"happiness_rank", "Happiness Score":"happiness", \
                                    "Economy (GDP per Capita)":"economy", "Family": "family", \
                                    'Health (Life Expectancy)':'health', \
                                    'Freedom': 'freedom', 'Trust (Government Corruption)':'trust' ,\
                                    'Generosity':'generosity', 'Dystopia Residual':'dystopia'})

hap_2017 = hap_2017.rename(columns={'Country':'country',"Happiness.Rank":"happiness_rank", "Happiness.Score":"happiness", \
                                    "Economy..GDP.per.Capita.":"economy", "Family": "family", \
                                    'Health..Life.Expectancy.':'health', \
                                    'Freedom': 'freedom', 'Trust..Government.Corruption.':'trust' ,\
                                    'Generosity':'generosity', 'Dystopia.Residual':'dystopia'})


happiness = hap_2015.append(hap_2016)
happiness = happiness.append(hap_2017)

In [6]:
happiness.head()

,country,dystopia,economy,family,freedom,generosity,happiness,happiness_rank,health,trust,year
0,Switzerland,2.51738,1.39651,1.34951,0.66557,0.29678,7.587,1,0.94143,0.41978,2015
1,Iceland,2.70201,1.30232,1.40223,0.62877,0.43630,7.561,2,0.94784,0.14145,2015
2,Denmark,2.49204,1.32548,1.36058,0.64938,0.34139,7.527,3,0.87464,0.48357,2015
3,Norway,2.46531,1.45900,1.33095,0.66973,0.34699,7.522,4,0.88521,0.36503,2015
4,Canada,2.45176,1.32629,1.32261,0.63297,0.45811,7.427,5,0.90563,0.32957,2015


In [8]:
countries = happiness['country'].unique()
countries.tolist()
countries = pd.DataFrame(countries, columns=['country'])

country_id = []
for i in range(len(countries)):
    country_id.append(i+1)

countries['country_id'] = country_id

regions = {'North America':["Canada","Mexico","US"],
           'South/Central America': ["Argentina","Brazil", "Chile", "Colombia", "Ecuador",                                   "Peru","Trinidad & Tobago", "Venezuela","Central America","Other Caribbean",                              "Other South America"], 
           'Europe':["Austria","Belgium","Bulgaria","Croatia","Cyprus","Czech Republic",                                "Denmark", "Estonia","Finland","France", "Germany","Greece","Hungary",                                  "Iceland", "Ireland","Italy",  "Latvia","Lithuania" ,"Luxembourg", "Netherlands",                         "North Macedonia", "Norway", "Poland","Portugal","Romania",                                                "Slovakia","Slovenia","Spain","Sweden" ,"Switzerland",                                                "Turkey", "Ukraine", "United Kingdom", "Other Europe"],
           'CIS':["Azerbaijan","Belarus","Kazakhstan","Russian Federation", "Turkmenistan","USSR", "Uzbekistan", "Other CIS"],
           'Middle East':["Iran","Iraq","Israel","Kuwait","Oman","Qatar", "Saudi Arabia", "United Arab Emirates", "Other Middle East"],
           'Africa':["Algeria","Egypt","Morocco","South Africa","Eastern Africa","Middle Africa", "Western Africa","Other Northern Africa" , "Other Southern Africa"],
           'Asia Pacific':["Australia","Bangladesh","China","China Hong Kong SAR", "India","Indonesia", "Japan","Malaysia","New Zealand","Pakistan","Philippines","Singapore", "South Korea","Sri Lanka","Taiwan","Thailand","Vietnam","Other Asia Pacific"]
            }

    # functions takes a dataframe with a 'Country' column and outputs with added 'Region' column
def label_regions(df):
    df['region'] = ""
    for region, countries in regions.items():
        df.loc[df["country"].isin(countries),"region"]=region
    return df

countries = label_regions(countries)
countries.head()

,country,country_id,region
0,Switzerland,1,Europe
1,Iceland,2,Europe
2,Denmark,3,Europe
3,Norway,4,Europe
4,Canada,5,North America


In [9]:
suicide_df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [10]:
group = suicide_df.groupby(['country'])
suicides_2015 = group.apply(lambda x: x[x['year'] == 2015]['suicides_no'].sum() )
suicides_2016 = group.apply(lambda x: x[x['year'] == 2016]['suicides_no'].sum() )

suicides_2015 = suicides_2015.to_frame()
suicides_2016 = suicides_2016.to_frame()

suicides_2015['year'] = 2015
suicides_2016['year'] = 2016

suicide = suicides_2015.append(suicides_2016)
suicide = suicide.rename( columns= {0:"suicides_no"} ) 
suicide.head()

,suicides_no,year
country,,
Albania,0,2015
Antigua and Barbuda,1,2015
Argentina,3073,2015
Armenia,74,2015
Aruba,0,2015


In [11]:
group = suicide_df.groupby(['country'])
pop_2015 = group.apply(lambda x: x[x['year'] == 2015]['population'].mean() )
pop_2016 = group.apply(lambda x: x[x['year'] == 2016]['population'].mean() )

pop_2015 = pop_2015.to_frame()
pop_2016 = pop_2016.to_frame()

pop_2015['year'] = 2015
pop_2016['year'] = 2016

population = pop_2015.append(pop_2016)
population = population.rename( columns= {0:"population"} ) 
population = pd.merge(population, countries, on="country")
population = population.drop( columns = ['country', 'region'])

population.head()

,population,year,country_id
0,NaN,2015,95
1,NaN,2016,95
2,3.308302e+06,2015,30
3,NaN,2016,30
4,2.329446e+05,2015,127


In [16]:
happiness = pd.merge(happiness, countries, on="country")
happiness = happiness.drop(columns=['country'])  
happiness.head()

,dystopia,economy,family,freedom,generosity,happiness,happiness_rank,health,trust,year,country_id,region
0,2.517380,1.39651,1.349510,0.66557,0.296780,7.587,1,0.94143,0.419780,2015,1,Europe
1,2.694630,1.52733,1.145240,0.58557,0.280830,7.509,2,0.86303,0.412030,2016,1,Europe
2,2.276716,1.56498,1.516912,NaN,0.290549,7.494,4,NaN,0.367007,2017,1,Europe
3,2.702010,1.30232,1.402230,0.62877,0.436300,7.561,2,0.94784,0.141450,2015,2,Europe
4,2.831370,1.42666,1.183260,0.56624,0.476780,7.501,3,0.86733,0.149750,2016,2,Europe


In [12]:
# coutries, population, happiness, and suicide tables ready to load to db

connection_string = "postgres:postgres@localhost:5432/etl_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# need to create etl_db to work

engine.table_names()